# MATCH (query)
Cypher works by matching patterns in the data. We retrieve data from the graph using the MATCH keyword. You can think of the MATCH clause as similar to the FROM clause in an SQL statement.

## match - NODE
```
MATCH (p:Person {name: 'Tom Hanks'})
RETURN  p.born
```

In Cypher, labels, property keys, and variables are case-sensitive. Cypher keywords are not case-sensitive.

Neo4j best practices include:
- Name labels using CamelCase.
- Name property keys and variables using camelCase.
- User UPPERCASE for Cypher keywords.

## match - RELATIONSHIPS
```
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m)
RETURN m.title
```
## match - MULTIPLE MATCH
```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.year > 2000
MATCH (m)<-[:DIRECTED]-(d:Person)
RETURN a.name, m.title, d.name
```

alternatives

```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie),
      (m)<-[:DIRECTED]-(d:Person)
WHERE m.year > 2000
RETURN a.name, m.title, d.name
```

## match - OPTIONAL MATCH

`OPTIONAL MATCH` matches patterns with your graph, just like `MATCH` does. 

**The difference is that if no matches are found, `OPTIONAL MATCH` will use nulls for missing parts of the pattern. `OPTIONAL MATCH` could be considered the Cypher equivalent of the outer join in SQL.**

Here is how you specify `OPTIONAL MATCH` in Cypher:
```
MATCH (m:Movie) WHERE m.title = "Kiss Me Deadly"
MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Actor)-[:ACTED_IN]->(rec)
RETURN rec.title, a.name
```


# FILTER 
## WHERE 
```
MATCH (p:Person)
WHERE p.name = 'Tom Hanks'
RETURN p.name
```

### where - OR
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.released = 2008 OR m.released = 2009
RETURN p, m
```
### where - AND
```
MATCH (p)-[:ACTED_IN]->(m)
WHERE p:Person AND m:Movie AND m.title='The Matrix'
RETURN p.name
```
### where - RANGE
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE 2000 <= m.released <= 2003
RETURN p.name, m.title, m.released
```
### where - IS NULL
```
MATCH (p:Person)
WHERE p.died IS NULL
AND p.born.year <= 1922
RETURN p.name, p.born, p.died
```
### where - IS NOT NULL
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name='Jack Nicholson' AND m.tagline IS NOT NULL
RETURN m.title, m.tagline
```
### where - STARTS WITH
When the property is a string type, you can filter by the starting characters in the string:
```
MATCH (m:Movie)
WHERE  m.title STARTS WITH 'Toy Story'
RETURN m.title, m.released
```
### where - ENDS WITH
And you can filter queries whose properties end with a set of characters:
```
MATCH (m:Movie)
WHERE  m.title ENDS WITH ' I'
RETURN m.title, m.released
```

### where - SUBSTRING - CONTAINS
Additionally, you can test if a substring is contained in a property:
```
MATCH (m:Movie)
WHERE  m.title CONTAINS 'River'
RETURN m.title, m.released
```

#### where - TOUPPER
```
MATCH (p:Person)
WHERE toUpper(p.name) ENDS WITH 'DEMILLE'
RETURN p.name
```
#### where - TOLOWER
```
MATCH (p:Person)
WHERE toLower(p.name) ENDS WITH 'demille'
RETURN p.name
```



### where - EXISTS
```
MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE NOT exists( (p)-[:DIRECTED]->(m) )
RETURN p.name, m.title
```
### where - IN LIST
```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE  'Neo' IN r.roles AND m.title='The Matrix'
RETURN p.name, r.roles
```

### where - INEQUALITY <>
You can also test **inequality of a property** using the `<>` predicate.
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name <> 'Tom Hanks'
AND m.title = 'Captain Phillips'
RETURN p.name
```
### where - LESS THAN - GREATER THAN
You can test both numbers and strings for values **less than** (`<`) or **greater than** (`>`) a value. Adding the equals sign will include the specified number within the predicate.

```
MATCH (m:Movie) WHERE m.title = 'Toy Story'
RETURN
    m.year < 1995 AS lessThan, //  Less than (false)
    m.year <= 1995 AS lessThanOrEqual, // Less than or equal(true)
    m.year > 1995 AS moreThan, // More than (false)
    m.year >= 1995 AS moreThanOrEqual // More than or equal (true)
```

### where - label existence
You can test for a label’s existence on a node using the `{alias}:{label}` syntax.

```
MATCH (p:Person)
WHERE  p.born.year > 1960
AND p:Actor
AND p:Director
RETURN p.name, p.born, labels(p)
```

# ORDERING RESULTS
## ordering - SINGLE
Whether you return results as nodes or as properties of nodes, you can specify a property value for the ordering. 
- Strings are ordered by their text values. 
- Boolean true comes before false when ordered. 
- Numeric data (including date and datetime properties) are ordered by their numeric value.

You specify `ORDER BY` in the `RETURN` clause where you specify the property for the ordering.

Here is an example:

```
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN p.name AS name,
p.born AS birthDate
ORDER BY p.born
```

The default ordering is ascending, but you can specify descending as follows:

```
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN p.name AS name, p.born AS birthDate
ORDER BY p.born DESC
```

## ordering - MULTIPLE
You can provide multiple sort expressions and the result will be sorted in that order.

```
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
OR p.name = 'Keanu Reeves'
RETURN  m.year, m.title
ORDER BY m.year DESC , m.title
```

There is no limit to the number of properties you can order by.

You can order by any property or expression that is within scope of the query. The ordering property or need not be explicitly returned, but can be used for the ordering.

# LIMIT
Although you can filter queries to reduce the number of results returned, you may also want to limit the number of results returned. This is useful if you have very large result sets and you only need to see the beginning or end of a set of ordered results. You can use the **LIMIT** keyword to specify the number of results returned.

```
MATCH (m:Movie)
WHERE m.released IS NOT NULL
RETURN m.title AS title,
m.released AS releaseDate
ORDER BY m.released DESC LIMIT 100
```

# SKIP
In an ordered result set, you may want to control what results are returned. This is useful in an application where pagination is required.
You can add a `SKIP` and `LIMIT` keyword to control what page of results are returned.

```
MATCH (p:Person)
WHERE p.born.year = 1980
RETURN  p.name as name,
p.born AS birthDate
ORDER BY p.born SKIP 40 LIMIT 10
```

# DISTINCT 
We can eliminate the duplication by specifying the DISTINCT keyword as follows:

```
MATCH (p:Person)-[:DIRECTED | ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN DISTINCT m.title, m.released
ORDER BY m.title
```
Using `DISTINCT` in the `RETURN` clause here means that rows with identical values will not be returned.

**You can use DISTINCT to eliminate duplication of:**

- rows returned (you have just learned this)
- property values
- nodes

# CASE 

```
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE p.name = 'Henry Fonda'
RETURN m.title AS movie,
CASE
WHEN m.year < 1940 THEN 'oldies'
WHEN 1940 <= m.year < 1950 THEN 'forties'
WHEN 1950 <= m.year < 1960 THEN 'fifties'
WHEN 1960 <= m.year < 1970 THEN 'sixties'
WHEN 1970 <= m.year < 1980 THEN 'seventies'
WHEN 1980 <= m.year < 1990 THEN 'eighties'
WHEN 1990 <= m.year < 2000 THEN 'nineties'
ELSE  'two-thousands'
END
AS timeFrame
```

# MERGE (create node and properties)

## Merge NODE

```
MERGE (p:Person {name: 'Michael Cain'})
```
```
MERGE (p:Person {name: 'Katie Holmes'})
MERGE (m:Movie {title: 'The Dark Knight'})
RETURN p, m
```
This code creates two nodes, each with a primary key property.


## Merge RELATIONSHIPS
Just like you can use MERGE to create nodes in the graph, you use MERGE to create relationships between two nodes. First you must have references to the two nodes you will be creating the relationship for. When you create a relationship between two nodes, it must have:

- Type
- Direction


```
MATCH (p:Person {name: 'Michael Cain'})
MATCH (m:Movie {title: 'The Dark Knight'})
MERGE (p)-[:ACTED_IN]->(m)
```

**Notice also that you need not specify direction in the MATCH pattern since the query engine will look for all nodes that are connected, regardless of the direction of the relationship.**

```
MERGE (p:Person {name: 'Chadwick Boseman'})
MERGE (m:Movie {title: 'Black Panther'})
MERGE (p)-[:ACTED_IN]-(m)
```
This code creates two nodes and a relationship between them.

**Note that in this MERGE clause where we create the relationships, we did not specify the direction of the relationship. By default, if you do not specify the direction when you create the relationship, it will always be assumed left-to-right.**

What MERGE does is create the node or relationship if it does not exist in the graph.
This code successfully creates the nodes and relationship:
```
MERGE (p:Person {name: 'Emily Blunt'})-[:ACTED_IN]->(m:Movie {title: 'A Quiet Place'})
RETURN p, m
```

# SET

## set - create properties
```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Cain' AND m.title = 'The Dark Knight'
SET r.roles = ['Alfred Penny']
RETURN p, r, m
```
```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Cain' AND m.title = 'The Dark Knight'
SET r.roles = ['Alfred Penny'], r.year = 2008
RETURN p, r, m
```
## set - update properties
```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Cain' AND m.title = 'The Dark Knight'
SET r.roles = ['Mr. Alfred Penny']
RETURN p, r, m
```
## set - remove properties (REMOVE OR SET)
You can remove or delete a property from a node or relationship by using the REMOVE keyword, or setting the property to null.

```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Cain' AND m.title = 'The Dark Knight'
REMOVE r.roles
RETURN p, r, m
```

Here we remove the born property from an actor:
```
MATCH (p:Person)
WHERE p.name = 'Gene Hackman'
SET p.born = null
RETURN p
```
**You should never remove the property that is used as the primary key for a node.**


## set - ON CREATE SET - ON MATCH SET

```
// Find or create a person with this name
MERGE (p:Person {name: 'McKenna Grace'})

// Only set the `createdAt` property if the node is created during this query
ON CREATE SET p.createdAt = datetime()

// Only set the `updatedAt` property if the node was created previously
ON MATCH SET p.updatedAt = datetime()

// Set the `born` property regardless
SET p.born = 2006

RETURN p
```

## set - CASE 
```
MATCH (p:Person)
SET p.born = CASE p.born WHEN "" THEN null ELSE date(p.born) END
WITH p
SET p.died = CASE p.died WHEN "" THEN null ELSE date(p.died) END
```

# DELETE 
In a Neo4j database you can delete:

- nodes
- relationships
- properties
- labels

To delete any data in the database, you must first retrieve it, then you can delete it.
## delete - NODE
```
MATCH (p:Person)
WHERE p.name = 'Jane Doe'
DELETE p
```
## delete - RELATIONSHIPS
To leave the Jane Doe node in the graph, but remove the relationship we retrieve the relationship and delete it.
```
MATCH (p:Person {name: 'Jane Doe'})-[r:ACTED_IN]->(m:Movie {title: 'The Matrix'})
DELETE r
RETURN p, m
```
If we attempt to delete the Jane Doe node, we will receive an error because it has relationships in the graph.

## delete - NODE WITH RELATIONSHIPS
```
MATCH (p:Person {name: 'Jane Doe'})
DETACH DELETE p
```

Do NOT delete all nodes and relationships in the graph as you need them for the next challenge!
```
MATCH (n)
DETACH DELETE n
```

# AGGREGATIONS

## aggregations - COUNT()
Cypher has a `count()` function that you can use to perform a **count of nodes, relationships, paths, rows during query processing**. When you aggregate in a Cypher statement, this means that the query must process all patterns in the `MATCH` clause to complete the aggregation to either return results or perform the next part of the query.

Here is an example:

```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE a.name = 'Tom Hanks'
RETURN a.name AS actorName,
count(*) AS numMovies
```
This query returns the **number of movies Tom Hanks acted in**.
When you aggregate in a Cypher statement, the query must process all patterns in the `MATCH` clause to complete the aggregation to return results. This is called **eager aggregation**.

**Aggregation in Cypher is different from aggregation in SQL**. 

In Cypher, you need not specify a **grouping key**. As soon as an aggregation function like `count()` is used, all non-aggregated result columns become grouping keys.

The grouping is implicitly done, based upon the fields in the `RETURN` clause.

- **If you specify `count(n)`, the graph engine calculates the number of non-null occurrences of n.** 
- **If you specify `count(*)`, the graph engine calculates the number of rows retrieved, including those with null values.**

# LIST
You can **return a list by specifying the square brackets**:

```
MATCH (p:Person)
RETURN p.name, [p.born, p.died] AS lifeTime
LIMIT 10
```

## list - COLLECT()
Cypher has a built-in aggregation function, `collect()` that enables you to **aggregate values into a list**. The value can be any expression, for instance a property value, a node, or result of a function or operation.

```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
RETURN a.name AS actor,
count(*) AS total,
collect(m.title) AS movies
ORDER BY total DESC LIMIT 10
```

You can eliminate duplication in the list as follows:

```
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.year = 1920
RETURN  collect( DISTINCT m.title) AS movies,
collect( a.name) AS actors
```

## list - SIZE()
The size() function returns the number of elements in a list.

Here we use size() to return the number of elements in the list:

```
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
RETURN actor.name, director.name,
size(collect(m)) AS collaborations,
collect(m.title) AS movies
```

## list - LIST COMPREHENSION
You can create a list by evaluating an expression that tests for list inclusion.

Here is an example:

```
MATCH (m:Movie)
RETURN m.title as movie,
[x IN m.countries WHERE x CONTAINS 'USA' OR x CONTAINS 'Germany']
AS country LIMIT 500
```
## list - PATTERN COMPREHENSION
Pattern comprehension is a very powerful way to create lists without changing the cardinality of the query. It behaves like an OPTIONAL MATCH combined with collect().

Here is an example:

```
MATCH (m:Movie)
WHERE m.year = 2015
RETURN m.title,
[(dir:Person)-[:DIRECTED]->(m) | dir.name] AS directors,
[(actor:Person)-[:ACTED_IN]->(m) | actor.name] AS actors
```

Notice that for pattern comprehension we specify the list with the square braces to include the pattern followed by the pipe character to then specify what value will be placed in the list from the pattern.
`[<pattern> | value]`

# MAP

A Cypher map is list of key/value pairs where each element of the list is of the format 'key': value. A node or relationship can have a property that is a map.

Using this map, we can return the value for one of its elements:

```
RETURN {Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}['Feb'] AS daysInFeb
```

Here we use the key, 'Feb' to access its value.

Alternatively, you can access a value with the `'.'` notation:

```
RETURN {Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 , Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}.Feb AS daysInFeb
```

## map - list of Keys
And you can return a list of keys of a map as follows:

```
RETURN keys({Jan: 31, Feb: 28, Mar: 31, Apr: 30 ,
May: 31, Jun: 30 ,Jul: 31, Aug: 31, Sep: 30,
Oct: 31, Nov: 30, Dec: 31}) AS months
```

## map - Map projections
The data is returned as rows of data where each row represents a JSON-style object for a node.

```
MATCH (p:Person)
WHERE p.name CONTAINS "Thomas"
RETURN p AS person
ORDER BY p.name ASC
```

This query returns all Person nodes that contain the string "Thomas". If you view the data returned as a table, it returns internal node information such as labels and identity, along with the property values.

Another way that you can return data is without the internal node information, that is, only property values.

```
MATCH (p:Person)
WHERE p.name CONTAINS "Thomas"
RETURN p { .* } AS person
ORDER BY p.name ASC
```

This query returns an object named person that contains all of the property values for the node. It does not contain any of the internal information for the node such as its labels or id.

Additionally, you can customize what properties you return in the objects.

```
MATCH (p:Person)
WHERE p.name CONTAINS "Thomas"
RETURN p { .name, .born } AS person
ORDER BY p.name
```

Here the person objects returned will include the name and born properties.

Here is an example, where we are adding information to the objects returned that are not part of the data in the graph.

```
MATCH (m:Movie)<-[:DIRECTED]-(d:Director)
WHERE d.name = 'Woody Allen'
RETURN m {.*, favorite: true} AS movie
```

In addition to returning all property values for each Woody Allen movie, we are returning a property of favorite with a value of true for each Movie object returned.

# UNWIND
The `UNWIND` clause expands the elements in genres list for the node as rows. With this data, it creates the Genre node using `MERGE`. With `MERGE`, it only creates the node if it does not already exist. Then it creates the relationship between the Movie node and the Genre node.

```
MATCH (m:Movie)
UNWIND m.genres AS genre
WITH m, genre
MERGE (g:Genre {name:genre})
MERGE (m)-[:IN_GENRE]->(g)
```

# LABELS
A best practice is to have at least one label for a node, but not more than four.

## labels - CREATE
```
MATCH (p:Person {name: 'Jane Doe'})
SET p:Developer
RETURN p
```
## labels - DELETE

To remove the newly-added label, Developer, you use the REMOVE clause. Run this code:
```
MATCH (p:Person {name: 'Jane Doe'})
REMOVE p:Developer
RETURN p
```

## labels - ADDING LABELS
add the Actor labels to the graph:
```
MATCH (p:Person)-[:ACTED_IN]->()
WITH DISTINCT p SET p:Actor
```
add the Director labels to the nodes.
```
MATCH (p:Person)-[:DIRECTED]->()
WITH DISTINCT p SET p:Director
```


## labels - labels()
You can test for a label’s existence on a node using the `{alias}:{label}` syntax.

```
MATCH (p:Person)
WHERE  p.born.year > 1960
AND p:Actor
AND p:Director
RETURN p.name, p.born, labels(p)
```

## labels - db.labels()

This code returns all node labels defined in the graph.
```
CALL db.labels()
```


# types()
A query with a pattern need not specify the relationship type in the query:
```
MATCH (p:Person)-[r]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
RETURN m.title AS movie, type(r) AS relationshipType
```

# Keys()

The properties for a node with a given label need not be the same. One way you can discover the properties for a node is to use the keys() function. This function returns a list of all property keys for a node.

```
MATCH (p:Person)
RETURN p.name, keys(p)
```
The results returned for each row include the name of the person, followed by the list of property keys for that node

# db.propertyKeys()

More generally, you can run this code to return all the property keys defined in the graph.
```
CALL db.propertyKeys()
```
Note that a property key remains in the graph, once it has been defined, even if there are currently no nodes or relationships that use that property key.


# DATE AND TIME
Cypher has these basic formats for storing date and time data.
```
RETURN date(), datetime(), time()
```
There are a number of other types of data such as `Time`, `LocalTime`, `LocalDateTime`, `Timestamp`, and `Duration` which are described in the **Temporal Functions** section of the Neo4j Cypher Manual.

## date-time - Extracting components of a date or datetime
You can **access the components of a date or datetime** property:

```
MATCH (x:Test {id: 1})
RETURN x.date.day, x.date.year,
x.datetime.year, x.datetime.hour,
x.datetime.minute
```

## date-time - Setting date values
You can use a string to **set a value for a date**:

```
MATCH (x:Test {id: 1})
SET x.date1 = date('2022-01-01'),
    x.date2 = date('2022-01-15')
RETURN x
```

You can set a Date property using a `<ISO-date>` string.

#### date-time - Setting datetime values
You can use a string to **set a value for a datetime**:

```
MATCH (x:Test {id: 1})
SET x.datetime1 = datetime('2022-01-04T10:05:20'),
    x.datetime2 = datetime('2022-04-09T18:33:05')
RETURN x
```

You can set a Datetime property using a `<ISO-datetime>` string.

#### date-time - durations
A duration is used to **determine the difference between two date/datetime values or to add or subtract a duration to a value**.

This code returns the **duration between date1 and date2** in the graph:

```
MATCH (x:Test {id: 1})
RETURN duration.between(x.date1,x.date2)
```

It returns a duration value that represents the **days and months and times** between the two values. In this case, the duration between date1 and date2 is 14 days.

We can return the **duration in days** between two datetime values:

```
MATCH (x:Test {id: 1})
RETURN duration.inDays(x.datetime1,x.datetime2).days
```

We can **add** a duration of 6 months:

```
MATCH (x:Test {id: 1})
RETURN x.date1 + duration({months: 6})
```

#### date-time -  APOC to format dates and times
The **APOC library** has many useful functions for working with all types of data.

Here is one way you can use APOC to **format a datetime**:

```
MATCH (x:Test {id: 1})
RETURN x.datetime as Datetime,
apoc.temporal.format( x.datetime, 'HH:mm:ss.SSSS')
AS formattedDateTime
```

Here is another example:

```
MATCH (x:Test {id: 1})
RETURN apoc.date.toISO8601(x.datetime.epochMillis, "ms")
AS iso8601
```

You can also use `apoc.temporal.toZonedTemporal()` for parsing arbitrary formatted temporal values with a format string.


# GRAPH MODELING
## GRAPH COMPONENTS
The Neo4j components that are used to define the graph data model are:

- **Nodes**
- **Labels**
- **Relationships**
- **Properties**

# Data modeling process
Here are the **steps to create a graph data model**:

- Understand the domain and define specific use cases (questions) for the application.
- Develop the initial graph data model:
- Model the nodes (entities).
- Model the relationships between nodes.
- Test the use cases against the initial data model.
- Create the graph (instance model) with test data using Cypher.
- Test the use cases, including performance against the graph.
- Refactor (improve) the graph data model due to a change in the key use cases or for performance reasons.
- Implement the refactoring on the graph and retest using Cypher.

## Types of models
When performing the graph data modeling process for an application, you will need at least two types of models:

- Data model
- Instance model

### Data model
**The data model describes the labels, relationships, and properties for the graph.** It does not have specific data that will be created in the graph.

Here is an example of a data model:

<img title="a title" alt="Alt text" src="./images/sample-data-model.png"  width="50%">

There is nothing that uniquely identifies a node with a given label. A graph data model, however is important because it defines the names that will be used for labels, relationship types, and properties when the graph is created and used by the application.

### Instance model
An important part of the graph data modeling process is to test the model against the use cases. To do this, you need to have a set of sample data that you can use to see if the use cases can be answered with the model.

Here is an example of an instance model:

<img title="a title" alt="Alt text" src="./images/sample-data-instance-model.png"  width="50%">

In this instance model, we have created some instances of Person and Movie nodes, as well as their relationships.

## Modeling Nodes
### node properties
**Node properties are used to**:

- **Uniquely identify a node.**
- **Answer specific details of the use cases for the application.**
- **Return data.**

## Modeling Relationships

When you create a relationship in Neo4j, a direction must either be specified explicitly or inferred by the left-to-right direction in the pattern specified. At runtime, during a query, direction is typically not required.

### Properties for relationships
Properties for a relationship are used to enrich how two nodes are related. When you define a property for a relationship, it is because your use cases ask a specific question about how two nodes are related, not just that they are related.

# REFACTOR PROPERTIES AS NODE
Here are the steps we use to refactor:

1. We take the property values for each Movie node and create a Language node.
2. Then we create the IN_LANGUAGE relationship between that Movie node and the Language node.
3. Finally, we remove the languages property from the Movie node.

This is the code to refactor the graph to turn the property values into nodes:

```
MATCH (m:Movie)
UNWIND m.languages AS language
WITH  language, collect(m) AS movies
MERGE (l:Language {name:language})
WITH l, movies
UNWIND movies AS m
WITH l,m
MERGE (m)-[:IN_LANGUAGE]->(l);
MATCH (m:Movie)
SET m.languages = null
```

There will only be one node with the language value of English and we remove the languages property from all Movie nodes. This eliminates a lot of duplication in the graph.

# PROFILE 
You can use the PROFILE keyword to see the performance for a query.

```
PROFILE MATCH (p:Person)-[:ACTED_IN]-()
WHERE p.born < '1950'
RETURN p.name
```
# EXPLAIN
```
EXPLAIN MATCH (m:Movie)
WHERE  m.title STARTS WITH 'Toy Story'
RETURN m.title, m.released
```
This query produces the execution plan where the first step is NodeIndexSeekByRange. In this case an index will be used because it is defined in the graph.

**The difference between using `EXPLAIN` and `PROFILE` is that `EXPLAIN` provides estimates of the query steps where `PROFILE` provides the exact steps and number of rows retrieved for the query.** 

# PATTERNS
A `pattern` is a **combination of nodes and relationships that is used to traverse the graph at runtime**. You can write queries that test whether a pattern exists in the graph.

Here is an example:
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
AND exists {(p)-[:DIRECTED]->(m)}
RETURN p.name, labels(p), m.title
```
## NON PATTERNS

There is a scenario where using `exists { }` for a pattern is useful. You use `NOT exists { }` to exclude patterns in the graph.

We want to find all the movies that Tom Hanks acted in, but did not direct.

```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE  p.name = 'Tom Hanks'
AND NOT exists {(p)-[:DIRECTED]->(m)}
RETURN  m.title
```


# IMPORT DATA
Cypher has a built-in clause, LOAD CSV for importing CSV files. If you have a JSON or XML file, you must use the APOC library to import the data, but you can also import CSV with APOC. And the Neo4j Data Importer enables you to import CSV data without writing any Cypher code.

**The types of data that you can store as properties in Neo4j include:**

- String
- Long (integer values)
- Double (decimal values)
- Boolean
- Date/Datetime
- Point (spatial)
- StringArray (comma-separated list of strings)
- LongArray (comma-separated list of integer values)
- DoubleArray (comma-separated list of decimal values)


There are two ways that you can import CSV data into Neo4j that you will learn in this course:

1. Using the Neo4j Data Importer.
2. Writing Cypher code to perform the import.

You must have one or more CSV files that represent the nodes and relationships that will be created in the graph. You must also have an existing Neo4j DBMS that is started. You typically start with a graph that has nothing in it.

## import data - CSV 

A CSV file represents rows of data that could be exported from relational and other databases, Web APIs, and other sources. When you are given CSV files, you must determine:

- Whether the CSV file will have header information, describing the names of the fields.
- What the delimiter will be for the fields in each row.

Including headers in the CSV file reduces syncing issues and is a recommended Neo4j best practice.

If the source CSV files use a different field terminator, you must specify the `FIELDTERMINATOR` in your Cypher LOAD CSV clause.

**If the headers do not correspond to the data representing the fields, you cannot load the data**. 

You must also know whether you can assume the use of the default delimiter ",", otherwise, you will need to use the `FIELDTERMINATOR` keyword along with LOAD CSV when you use Cypher to import the data.

You should have a local copy of the CSV files so you can inspect the data in them. In fact, **when using the Neo4j Data Importer you will need a local copy of the CSV files**.


### NORMALIZED DATA
Data normalization is common in relational models. **This enables you to have CSV files that correspond to a relational table where an ID is used to identify the relationships**.


### DE-NORMALIZED DATA
With de-normalized data, **the data is represented by multiple rows corresponding to the same entity**, which will be loaded as a node. The difference, however, is that de-normalized data typically **represents data from multiple tables in the RDBMS**.

### ID UNIQUE
When you load data from CSV files, you rely heavily upon the IDs specified in the file. **A Neo4j best practice is to use an ID as a unique property value for each node.** If the IDs in your CSV file are not unique for the same entity (node), you will have problems when you load the data and try to create relationships between existing nodes.

## import data - NEO4J DATA-IMPORTER

You can import or export your mappings to a JSON file or to a ZIP file if you also want to include the CSV files.

The Data Importer is good for small to medium datasets(**less than 1M rows**), but if you have a large dataset, you will need to use Cypher to import the data.

The Data Importer is a generalized app as you saw that creates all properties in the graph as strings, integers, decimals, datetimes, or boolean, and you need to possibly post-process or refactor the graph after the import.

### Requirements for using the Data Importer
- You must use **CSV** files for import.
- CSV files must reside **on your local system** so you can load them into the graph app.
- CSV data must be **clean** (you learned this in an earlier lesson).
- **IDs must be unique for all nodes** you will be creating.
- The CSV file must have **headers**.
- The DBMS must be **started**.


- Step 1: Placing the CSV file(s) on your local system and ensure they have headers and are clean.
- Step 2: Opening the Neo4j Data Importer
- Step 3: Loading the CSV files on your local system into the graph app
- Step 4: Examining the CSV header names used in the CSV files
  - You will **examine the first rows** of each CSV file to determine:
  - Files to be used to create nodes.
  - Files to be used to create relationships.
  - How IDs are used to uniquely identify data.
- Step 5: Adding a node
- Step 6: Defining mapping details for the node
- Step 7: Creating the relationships between nodes
- Step 8: Defining the mapping details for the relationship
- Step 9: Performing the import
  - Import the data.
  - View the import results.
 - Step 10: Viewing the imported data in Neo4j Browser


## import data - IMPORT WITH CYPHER (CALL OR LOAD)

In Cypher, by default, the execution of your code is a single transaction. In order to process large CSV imports, you may need to break up the execution of the Cypher into multiple transactions.

**One advantage of using Cypher for loading your CSV data is that you can perform the type transformations and some of the "refactoring" during the import.**

**You can use this code structure to import a large dataset:**

```
CALL {
// add data to the graph for each row
}
```

If the CSV data is too large, you may need to modify as follows:

```
LOAD CSV WITH HEADERS
FROM 'https://xxx.com/xxx/large-file.csv'
AS row
CALL {
  WITH row
  // create data in the graph
} IN TRANSACTIONS
```
### IMPORT MOVIE ADN GENRE
verify that the movie data is being properly transformed from the CSV file:

```
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing/2-movieData.csv'
AS row
//process only Movie rows
WITH row WHERE row.Entity = "Movie"
RETURN
toInteger(row.tmdbId),
toInteger(row.imdbId),
toInteger(row.movieId),
toFloat(row.imdbRating),
row.released,
row.title,
toInteger(row.year),
row.poster,
toInteger(row.runtime),
split(coalesce(row.countries,""), "|"),
toInteger(row.imdbVotes),
toInteger(row.revenue),
row.plot,
row.url,
toInteger(row.budget),
split(coalesce(row.languages,""), "|"),
split(coalesce(row.genres,""), "|")
LIMIT 10
```

read the CSV data and create the Movie and Genre nodes:


```
CALL {
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing/2-movieData.csv'
AS row
//process only Movie rows
WITH row WHERE row.Entity = "Movie"
MERGE (m:Movie {movieId: toInteger(row.movieId)})
ON CREATE SET
m.tmdbId = toInteger(row.tmdbId),
m.imdbId = toInteger(row.imdbId),
m.imdbRating = toFloat(row.imdbRating),
m.released = datetime(row.released),
m.title = row.title,
m.year = toInteger(row.year),
m.poster = row.poster,
m.runtime = toInteger(row.runtime),
m.countries = split(coalesce(row.countries,""), "|"),
m.imdbVotes = toInteger(row.imdbVotes),
m.revenue = toInteger(row.revenue),
m.plot = row.plot,
m.url = row.url,
m.budget = toInteger(row.budget),
m.languages = split(coalesce(row.languages,""), "|")
WITH m,split(coalesce(row.genres,""), "|") AS genres
UNWIND genres AS genre
WITH m, genre
MERGE (g:Genre {name:genre})
MERGE (m)-[:IN_GENRE]->(g)
}
```




### IMPORT PERSON

verify that the person data is being property transformed from the CSV file:
```
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing/2-movieData.csv'
AS row
WITH row WHERE row.Entity = "Person"
RETURN
toInteger(row.tmdbId),
toInteger(row.imdbId),
row.bornIn,
row.name,
row.bio,
row.poster,
row.url,
CASE row.born WHEN "" THEN null ELSE datetime(row.born) END,
CASE row.died WHEN "" THEN null ELSE datetime(row.died) END
LIMIT 10
```

create nodes

```
CALL {
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing/2-movieData.csv'
AS row
WITH row WHERE row.Entity = "Person"
MERGE (p:Person {tmdbId: toInteger(row.tmdbId)})
ON CREATE SET
p.imdbId = toInteger(row.imdbId),
p.bornIn = row.bornIn,
p.name = row.name,
p.bio = row.bio,
p.poster = row.poster,
p.url = row.url,
p.born = CASE row.born WHEN "" THEN null ELSE date(row.born) END,
p.died = CASE row.died WHEN "" THEN null ELSE date(row.died) END
}
```
### import relationships

```
CALL {
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing/2-movieData.csv'
AS row
WITH row WHERE row.Entity = "Join" AND row.Work = "Acting"
MATCH (p:Person {tmdbId: toInteger(row.tmdbId)})
MATCH (m:Movie {movieId: toInteger(row.movieId)})
MERGE (p)-[r:ACTED_IN]->(m)
ON CREATE
SET r.role = row.role
SET p:Actor
}
```

# APOC

## apoc - NODE TYPE PROPERTIES

You can use this Cypher code to show the stored types for the node properties in the graph:
```
CALL apoc.meta.nodeTypeProperties()
YIELD nodeType, propertyName, propertyTypes
```

## apoc - REL TYPE PROPERTIES

And you can do the same type of investigation of types for relationship properties:
```
CALL apoc.meta.relTypeProperties()
YIELD relType, propertyName, propertyTypes
```


# TRANSFORM 
## transform - STRING

```
MATCH (p:Person)
SET p.born = CASE p.born WHEN "" THEN null ELSE date(p.born) END
WITH p
SET p.died = CASE p.died WHEN "" THEN null ELSE date(p.died) END
```

## transform - STRINGS TO LIST

**A multi-value property is a property that can hold one or more values**. This type of data in Neo4j is represented as a list. 

**All values in a list must have the same type**. 

Transforming multi-value fields as lists can be done as follows where we use two Cypher built-in functions to help us:

```
MATCH (m:Movie)
SET m.countries = split(coalesce(m.countries,""), "|"),
m.languages = split(coalesce(m.languages,""), "|"),
m.genres = split(coalesce(m.genres,""), "|")
```

- `coalesce()` returns an empty string if the entry in m.countries is null. 
- `split()` identifies each element in the multi-value field where the "|" character is the separator and create a list of each element.


# CONSTRAINTS
## constraints - UNIQUE CONSTRAINTS
When you used the Data Importer, it automatically created the uniqueness constraints in the graph for the unique IDs you specified when you imported the data.

**A best practice is to always have a unique ID for every type of node in the graph.** 

Having a uniqueness constraint defined helps with performance when creating nodes and also for queries. The MERGE clause looks up nodes using the property value defined for the constraint. With a constraint, it is a quick lookup and if the node already exists, it is not created.

## constraints - SHOW CONSTRAINTS
You can view the constraints defined in the graph with the `SHOW CONSTRAINTS` command in Neo4j Browser:

## constraints - CREATE UNIQUENESS CONSTRAINT

Here is the **code we use to create this uniqueness constraint for the name property of Genre nodes**:

```
CREATE CONSTRAINT Genre_name IF NOT EXISTS
FOR (x:Genre)
REQUIRE x.name IS UNIQUE
```

# DB SCHEMA
View the schema.
```
CALL db.schema.visualization
```
You can view the **property types for nodes in the graph** 
```
CALL db.schema.nodeTypeProperties()
```
You can view the **property types for relationships in the graph** 
```
CALL db.schema.relTypeProperties()
```

# QUERY TRAVERSAL
## BASIC QUERY TRAVERSAL
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Eminem'
RETURN  m.title AS movies
```

Here is what happens when this query executes:

<img src="./images/traverse-Eminem-movies.gif" width="600" align="center"  width="50%">

1. The Eminem Person node is retrieved.
2. Then the first ACTED_IN relationship is traversed to retrieve the Movie node for 8 Mile.
3. Then the second ACTED_IN relationship is traversed to retrieve the Movie node for Hip Hop Witch, Da.
4. The title property is retrieved so that the results can be returned.


## QUERY TRAVERSAL MULTIPLE MATCH
```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Eminem'
MATCH (allActors:Person)-[:ACTED_IN]->(m)
RETURN m.title AS movie, collect(allActors.name) AS allActors
```

Here is what happens when this query executes:

<img src="./images/traverse-multiple_match.gif" width="600" align="center"  width="50%">

1. For the first MATCH clause in the query, the Eminem Person node is retrieved.
2. Then the first ACTED_IN relationship is traversed to retrieve the Movie node for 8 Mile.
3. The second MATCH clause in the query is then executed.
4. Each ACTED_IN relationship to the same 8 Mile movie is traversed to retrieve all actors, including the relationship to the Eminem node.
5. Then the query returns back to the first MATCH clause to traverse the ACTED_IN relationship to the Hip Hop  Witch, Da movie.
6. The second MATCH clause in the query is then executed.
7. Each ACTED_IN relationship to the same Hip Hop Witch, Da movie is traversed to retrieve all actors.

## SHORTEST PATH

**In Neo4j uniqueness of relationships is always adhered to. That is, `there will never be two relationships of the same type and direction between two nodes`. This enables Neo4j to avoid cycles or infinite loops in graph traversal.**

```
MATCH p = shortestPath((p1:Person)-[*]-(p2:Person))
WHERE p1.name = "Eminem"
AND p2.name = "Charlton Heston"
RETURN  p
```

This query calculates and returns the shortest path between the Eminem node and the Charlton Heston node in the graph. **Because nothing is specified for the relationship type, it finds the shortest path regardless of the relationship type**.

You can also **limit the relationship types** to a specific relationship:

```
MATCH p = shortestPath((p1:Person)-[:ACTED_IN*]-(p2:Person))
WHERE p1.name = "Eminem"
AND p2.name = "Charlton Heston"
RETURN  p
```

**For `shortestPath()` and `allShortestPaths()` you can provide an upper bound on the length of the path(s), but not a lower bound.**

### exact hops

Retrieve **all Person nodes that are exactly four hops away from Eminem** using the `ACTED_IN` relationship.

```
MATCH (p:Person {name: 'Eminem'})-[:ACTED_IN*4]-(others:Person)
RETURN  others.name
```

### up to hops number
Suppose you want to retrieve **all Person nodes that are up to four hops away from Eminem** using the `ACTED_IN` relationship.

```
MATCH (p:Person {name: 'Eminem'})-[:ACTED_IN*1..4]-(others:Person)
RETURN  others.name
```

# WITH - SCOPE VARIABLE
## with - redefine scope
```
WITH  'toy story' AS mt, 'Tom Hanks' AS actorName
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH m, toLower(m.title) AS movieTitle
WHERE p.name = actorName
AND movieTitle CONTAINS mt
RETURN m.title AS movies, movieTitle
```

A WITH clause is used to define or redefine the scope of variables. Because we want to redefine what is used for the WHERE clause, we add a new WITH clause. This creates a new scope for the remainder of the query so that m and movieTitle can be used to return values.

## with - limit results
Use WITH to limit how many m nodes are used later in the query

```
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m  LIMIT 2
// possibly do more with the two m nodes
RETURN m.title AS movies
```
## with - ordering
Use WITH to order nodes that are used later in the query

```
WITH  'Tom Hanks' AS theActor
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = theActor
WITH m ORDER BY m.year LIMIT 5
// possibly do more with the five m nodes in a particular order
RETURN m.title AS movies, m.year AS yearReleased
```

## with - map projections
customize the properties of nodes that are returned. Using a map projection, you can specify which properties are returned.

```
MATCH (n:Movie)
WHERE n.imdbRating IS NOT NULL
AND n.poster IS NOT NULL
WITH n {
  .title,
  .year,
  .languages,
  .plot,
  .poster,
  .imdbRating,
  directors: [ (n)<-[:DIRECTED]-(d) | d { tmdbId:d.imdbId, .name } ]
}
ORDER BY n.imdbRating DESC LIMIT 4
RETURN collect(n)
```
## with - unwind

```
MATCH (m:Movie)
UNWIND m.languages AS lang
WITH m, trim(lang) AS language
// this automatically, makes the language distinct because it's a grouping key
WITH language, collect(m.title) AS movies
RETURN language, movies[0..10]
```

This query:

1. Retrieves all Movie nodes.
2. For each Movie node, it unwinds the languages list to create a list called lang. Notice that we use the trim() function to ensure there are no extraneous whitespace characters in the language name.
3. Then we use the element of the list to find all Movies that use that language.
4. Finally, we return a row that contains each language name and the list of up to 10 movie titles for that language.


# SUB-QUERIES
A subquery is a set of Cypher statements that execute within their own scope. A subquery is typically called from an outer enclosing query. Using a subquery, you can limit the number of rows that need to be processed.

Here are some important things to know about a subquery:

- A subquery returns values referred to by the variables in the RETURN clause.
- A subquery cannot return variables with the same name used in the enclosing query.
- You must explicitly pass in variables from the enclosing query to a subquery.


## subquery - CALL
In a CALL clause, you specify a query that can return data from the graph or derived from the graph. A set of nodes returned in the CALL clause can be used by the enclosing query.

```
CALL {
   MATCH (m:Movie) WHERE m.year = 2000
   RETURN m ORDER BY m.imdbRating DESC LIMIT 10
}
MATCH  (:User)-[r:RATED]->(m)
RETURN m.title, avg(r.rating)
```
### subquery - CALL - passing varibale
Here is an example where the subquery is executed after the initial query and the enclosing query passes a variable, m into the subquery.

```
MATCH (m:Movie)
CALL {
    WITH m
    MATCH (m)<-[r:RATED]-(u:User)
     WHERE r.rating = 5
    RETURN count(u) AS numReviews
}
RETURN m.title, numReviews
ORDER BY numReviews DESC
```

## subquery - UNION
As your queries become more complex, you may need to combine the results of multiple queries. You can do so with UNION. With UNION, the queries you are combining must return the same number of properties or data

```
MATCH (m:Movie) WHERE m.year = 2000
RETURN {type:"movies", theMovies: collect(m.title)} AS data
UNION ALL
MATCH (a:Actor) WHERE a.born.year > 2000
RETURN { type:"actors", theActors: collect(DISTINCT a.name)} AS data
```

Because both queries return a variable named Data, we can combine the results using UNION ALL.

**`UNION ALL` returns all results which is more efficient on memory but can lead to duplicates. UNION returns distinct results.**

Results of a UNION cannot be directly post-processed. But if you wrap a UNION in a subquery, you can then further process the results.

```
MATCH (p:Person)
WITH p LIMIT 100
CALL {
  WITH p
  OPTIONAL MATCH (p)-[:ACTED_IN]->(m:Movie)
  RETURN m.title + ": " + "Actor" AS work
UNION
  WITH p
  OPTIONAL MATCH (p)-[:DIRECTED]->(m:Movie)
  RETURN m.title+ ": " +  "Director" AS work
}
RETURN p.name, collect(work)
```


# PARAMETERS
In your Cypher statements, a parameter name begins with the `$` symbol.

```
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = $actorName
RETURN m.released AS releaseDate,
m.title AS title
ORDER BY m.released DESC
```

**You will not be able to run this code yet in Neo4j Browser because you have not yet set a value for the parameter.**

## parameters - set params
You can set values for Cypher parameters that will be in effect during your Neo4j Browser session.

```
:param actorName: 'Tom Hanks'
```

Special consideration should be made when setting integer values in a Neo4j Browser session.
```
:param number: 10
```
The Browser will output the following result, with number cast as a `float`.

Instead, to force the number to be an integer, you can use the `⇒` operator.

```
:param number=> 10
```
## parameters - set multiple params

You can also use the `JSON-style` syntax to set all of the parameters in your Neo4j Browser session. The values you can specify in this object are **numbers**, **strings**, and **booleans**. 

```
:params {actorName: 'Tom Cruise', movieName: 'Top Gun'}
```

## parameters - Viewing parameters
If you want to **view the current parameters and their values**, simply type:

```
:params
```
## parameters - remove parameters
If you want to **clear all parameters**, you can simply type:

```
:params {}
```